# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'QETA/ANSG CGAASOKR (BDESMTN ELL)L',
 'calculationPrice': 'close',
 'open': 127.36,
 'openTime': 1671500846131,
 'openSource': 'laiffoci',
 'close': 133.97,
 'closeTime': 1683424959075,
 'closeSource': 'acffoili',
 'high': 128.88,
 'highTime': 1690369645896,
 'highSource': 'lp5rme auite cd1 edieny',
 'low': 124.2,
 'lowTime': 1641330766169,
 'lowSource': 'drm15utla  einpee dciey',
 'latestPrice': 131.48,
 'latestSource': 'Close',
 'latestTime': 'March 1, 2021',
 'latestUpdate': 1644016458769,
 'latestVolume': 120808900,
 'iexRealtimePrice': 133.01,
 'iexRealtimeSize': 3,
 'iexLastUpdated': 1664808179726,
 'delayedPrice': 130.16,
 'delayedPriceTime': 1693133682929,
 'oddLotDelayedPrice': 131.52,
 'oddLotDelayedPriceTime': 1681793959630,
 'extendedPrice': 133.68,
 'extendedChange': 0.96,
 'extendedChangePercent': 0.00766,
 'extendedPriceTime': 1638641843026,
 'previousClose': 122.86,
 'previousVolume': 172489098,
 'chang

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

36.37

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

Ticker   Price Price-to-Earnings Ratio Number of Shares to Buy
0        A  126.32                   55.04                     N/A
1      AAL   21.42                   -1.16                     N/A
2      AAP  163.22                    23.2                     N/A
3     AAPL  129.21                   36.19                     N/A
4     ABBV  112.50                   40.82                     N/A
..     ...     ...                     ...                     ...
500    YUM  108.39                   36.53                     N/A
501    ZBH  163.60                 -253.76                     N/A
502   ZBRA  535.84                      56                     N/A
503   ZION   55.69                    8.81                     N/A
504    ZTS  161.54                   47.49                     N/A

[505 rows x 4 columns]

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

Ticker   Price Price-to-Earnings Ratio Number of Shares to Buy
0     NRG   42.07                     2.5                     475
1     BIO  612.40                     4.8                      32
2     AIV    4.88                    4.83                    4098
3     ALL  112.63                    6.44                     177
4     UNM   27.60                    7.01                     724
5     AFL   50.84                     7.4                     393
6    EBAY   59.09                    7.62                     338
7     CPB   47.04                    7.99                     425
8     KIM   19.41                    8.49                    1030
9      KR   34.02                    8.72                     587
10   ZION   55.69                    8.81                     359
11     CE  146.92                    8.82                     136
12    PHM   46.75                    9.16                     427
13    SRE  121.40                    9.22                     164
14     CI  218.41                    9.54                      91
15    CTL   11.00                   10.09                    1818
16    HII  183.31                   10.44                     109
17    PNC  179.50                   10.52                     111
18    LEN   84.20                   10.54                     237
19    PGR   90.96                   10.57                     219
20    MET   59.71                   10.67                     334
21    DHI   78.84                   10.77                     253
22   DISH   33.01                   11.02                     605
23    DGX  121.26                   11.19                     164
24    HIG   53.38                   11.19                     374
25     BK   44.10                    11.4                     453
26   BIIB  274.74                    11.5                      72
27    CAH   53.66                   11.65                     372
28    WHR  202.60                   11.99                      98
29    PFG   60.10                   12.04                     332
30    UHS  132.30                   12.07                     151
31    MAS   56.09                   12.32                     356
32     GM   53.25                   12.53                     375
33    STT   77.19                   12.55                     259
34    TSN   71.46                   12.73                     279
35    CVS   72.55                   12.93                     275
36    ETR   91.22                   12.98                     219
37   INTC   65.01                   13.22                     307
38     WU   24.75                   13.29                     808
39     VZ   56.19                   13.48                     355
40    MGM   40.20                   13.63                     497
41    SEE   43.75                   13.74                     457
42     MS   79.55                   13.89                     251
43   CINF  105.17                   13.94                     190
44     GS  330.93                   13.97                      60
45      C   72.63                   14.07                     275
46   FOXA   36.04                   14.29                     554
47   HOLX   77.26                    14.3                     258
48    LMT  335.73                   14.34                      59
49    GIS   56.77                   14.56                     352

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

Ticker     Price Number of Shares to Buy Price-to-Earnings Ratio  \
40     AON   234.850                     N/A                    None   
71   BRK.B   251.370                     N/A                    None   
118    CTL    11.000                     N/A                      10   
136  DISCK    49.740                     N/A                    None   
165   ETFC    50.750                     N/A                   14.37   
168   EVRG    55.400                     N/A                    None   
190    FOX    34.040                     N/A                    None   
192    FRC   180.960                     N/A                    None   
204   GOOG  2173.110                     N/A                    None   
326    MYL    15.954                     N/A                   31.45   
327    NBL     8.610                     N/A                   -0.76   
348    NWS    23.920                     N/A                    None   
363    PEG    55.330                     N/A                    None   
442   TROW   168.590                     N/A                    None   
452     UA    19.230                     N/A                    None   
454    UAL    55.800                     N/A                    None   
498    XRX    26.710                     N/A                    None   

    PE Percentile Price-to-Book Ratio PB Percentile Price-to-Sales Ratio  \
40            N/A                None           N/A                 None   
71            N/A                None           N/A                 None   
118           N/A                None           N/A                 None   
136           N/A                None           N/A                 None   
165           N/A                None           N/A                 None   
168           N/A                None           N/A                 None   
190           N/A                None           N/A                 None   
192           N/A                None           N/A                 None   
204           N/A                None           N/A                 None   
326           N/A                None           N/A                 None   
327           N/A                None           N/A                 None   
348           N/A                None           N/A                 None   
363           N/A                None           N/A                 None   
442           N/A                None           N/A                 None   
452           N/A                None           N/A                 None   
454           N/A                None           N/A                 None   
498           N/A                None           N/A                 None   

    PS Percentile  EV/EBITDA EV/EBITDA Percentile  EV/GP EV/GP Percentile  \
40            N/A        NaN                  N/A    NaN              N/A   
71            N/A        NaN                  N/A    NaN              N/A   
118           N/A        NaN                  N/A    NaN              N/A   
136           N/A        NaN                  N/A    NaN              N/A   
165           N/A        NaN                  N/A    NaN              N/A   
168           N/A        NaN                  N/A    NaN              N/A   
190           N/A        NaN                  N/A    NaN              N/A   
192           N/A        NaN                  N/A    NaN              N/A   
204           N/A        NaN                  N/A    NaN              N/A   
326           N/A        NaN                  N/A    NaN              N/A   
327           N/A        NaN                  N/A    NaN              N/A   
348           N/A        NaN                  N/A    NaN              N/A   
363           N/A        NaN                  N/A    NaN              N/A   
442           N/A        NaN                  N/A    NaN              N/A   
452           N/A        NaN                  N/A    NaN              N/A   
454           N/A        NaN                  N/A    NaN              N/A   
498           N/A        NaN 

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

Empty DataFrame
Columns: [Ticker, Price, Number of Shares to Buy, Price-to-Earnings Ratio, PE Percentile, Price-to-Book Ratio, PB Percentile, Price-to-Sales Ratio, PS Percentile, EV/EBITDA, EV/EBITDA Percentile, EV/GP, EV/GP Percentile, RV Score]
Index: []

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.849505
1      0.150495
2      0.447525
3      0.706931
4      0.756436
         ...   
500    0.722772
501    0.009901
502    0.861386
503    0.178218
504    0.818812
Name: PE Percentile, Length: 505, dtype: object
0      0.730693
1      0.047525
2      0.419802
3      0.954455
4      0.883168
         ...   
500    0.045545
501     0.39604
502    0.853465
503    0.128713
504    0.920792
Name: PB Percentile, Length: 505, dtype: object
0      0.811881
1      0.073267
2      0.117822
3       0.80099
4      0.609901
         ...   
500    0.735644
501     0.69604
502    0.755446
503    0.384158
504    0.924752
Name: PS Percentile, Length: 505, dtype: object
0      0.877228
1      0.033663
2      0.217822
3      0.790099
4      0.415842
         ...   
500    0.734653
501    0.869307
502    0.893069
503    0.063366
504    0.875248
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.79604
1       0.10495
2      0.108911
3      0.922772
4      0.607921
         ...   
5

Ticker   Price Number of Shares to Buy  Price-to-Earnings Ratio  \
0        A  128.32                     N/A                    54.48   
1      AAL   21.72                     N/A                    -1.15   
2      AAP  165.01                     N/A                    23.77   
3     AAPL  134.01                     N/A                    36.22   
4     ABBV  111.42                     N/A                    40.28   
..     ...     ...                     ...                      ...   
500    YUM  109.62                     N/A                    37.14   
501    ZBH  168.10                     N/A                  -254.41   
502   ZBRA  514.09                     N/A                    57.30   
503   ZION   55.52                     N/A                     8.84   
504    ZTS  160.64                     N/A                    46.83   

    PE Percentile  Price-to-Book Ratio PB Percentile  Price-to-Sales Ratio  \
0        0.849505                 7.73      0.730693                7.1700   
1        0.150495                -1.95      0.047525                0.7836   
2        0.447525                 3.09      0.419802                1.0500   
3        0.706931                32.16      0.954455                7.0100   
4        0.756436                15.25      0.883168                4.3000   
..            ...                  ...           ...                   ...   
500      0.722772                -3.98      0.045545                5.6000   
501      0.009901                 2.90       0.39604                4.9800   
502      0.861386                12.65      0.853465                6.0000   
503      0.178218                 1.22      0.128713                2.5000   
504      0.818812                20.11      0.920792               11.2500   

    PS Percentile  EV/EBITDA EV/EBITDA Percentile      EV/GP EV/GP Percentile  \
0        0.811881  32.751629             0.877228  13.461088          0.79604   
1        0.073267  -4.386988             0.033663   2.264593          0.10495   
2        0.117822  10.730406             0.217822   2.412225         0.108911   
3         0.80099  24.884904             0.790099  18.899728         0.922772   
4        0.609901  14.402806             0.415842   9.104474         0.607921   
..            ...        ...                  ...        ...              ...   
500      0.735644  22.699450             0.734653   9.611493         0.653465   
501       0.69604  30.525738             0.869307   8.282914         0.524752   
502      0.755446  37.453665             0.893069  14.283792         0.829703   
503      0.384158   6.544386             0.063366   2.878890         0.128713   
504      0.924752  31.965440             0.875248  16.764964          0.89505   

    RV Score  
0        N/A  
1        N/A  
2        N/A  
3        N/A  
4        N/A  
..       ...  
500      N/A  
501      N/A  
502      N/A  
503      N/A  
504      N/A  

[505 rows x 14 columns]

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

Ticker   Price Number of Shares to Buy  Price-to-Earnings Ratio  \
0        A  128.32                     N/A                    54.48   
1      AAL   21.72                     N/A                    -1.15   
2      AAP  165.01                     N/A                    23.77   
3     AAPL  134.01                     N/A                    36.22   
4     ABBV  111.42                     N/A                    40.28   
..     ...     ...                     ...                      ...   
500    YUM  109.62                     N/A                    37.14   
501    ZBH  168.10                     N/A                  -254.41   
502   ZBRA  514.09                     N/A                    57.30   
503   ZION   55.52                     N/A                     8.84   
504    ZTS  160.64                     N/A                    46.83   

    PE Percentile  Price-to-Book Ratio PB Percentile  Price-to-Sales Ratio  \
0        0.849505                 7.73      0.730693                7.1700   
1        0.150495                -1.95      0.047525                0.7836   
2        0.447525                 3.09      0.419802                1.0500   
3        0.706931                32.16      0.954455                7.0100   
4        0.756436                15.25      0.883168                4.3000   
..            ...                  ...           ...                   ...   
500      0.722772                -3.98      0.045545                5.6000   
501      0.009901                 2.90       0.39604                4.9800   
502      0.861386                12.65      0.853465                6.0000   
503      0.178218                 1.22      0.128713                2.5000   
504      0.818812                20.11      0.920792               11.2500   

    PS Percentile  EV/EBITDA EV/EBITDA Percentile      EV/GP EV/GP Percentile  \
0        0.811881  32.751629             0.877228  13.461088          0.79604   
1        0.073267  -4.386988             0.033663   2.264593          0.10495   
2        0.117822  10.730406             0.217822   2.412225         0.108911   
3         0.80099  24.884904             0.790099  18.899728         0.922772   
4        0.609901  14.402806             0.415842   9.104474         0.607921   
..            ...        ...                  ...        ...              ...   
500      0.735644  22.699450             0.734653   9.611493         0.653465   
501       0.69604  30.525738             0.869307   8.282914         0.524752   
502      0.755446  37.453665             0.893069  14.283792         0.829703   
503      0.384158   6.544386             0.063366   2.878890         0.128713   
504      0.924752  31.965440             0.875248  16.764964          0.89505   

     RV Score  
0    0.813069  
1     0.08198  
2    0.262376  
3     0.83505  
4    0.654653  
..        ...  
500  0.578416  
501  0.499208  
502  0.838614  
503  0.176634  
504  0.886931  

[505 rows x 14 columns]

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:1000000


In [20]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/home/actserv/Documents/upwork_data_science/env/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Ticker   Price Number of Shares to Buy  Price-to-Earnings Ratio  \
0     UNM   27.60                     724                     7.09   
1     MCK  177.32                     112                    -6.99   
2     FTI    8.63                    2317                    -0.67   
3     AIG   47.31                     422                    -6.70   
4     MET   61.15                     327                    10.76   
5     ALL  115.17                     173                     6.35   
6     DXC   27.09                     738                    -2.38   
7     AAL   21.72                     920                    -1.15   
8     ABC  106.09                     188                    -6.55   
9       F   11.99                    1668                   -38.89   
10    HIG   52.57                     380                    11.25   
11     BA  225.02                      88                   -10.85   
12    PFG   59.29                     337                    11.57   
13    CVS   69.34                     288                    13.18   
14    AIZ  128.10                     156                    19.11   
15    AFL   49.08                     407                     7.50   
16      T   28.31                     706                   -39.01   
17    TAP   45.02                     444                   -10.71   
18   PRGO   42.71                     468                  -909.75   
19     KR   34.34                     582                     9.02   
20    TRV  157.39                     127                    14.80   
21    UHS  130.10                     153                    12.08   
22    HPQ   29.79                     671                    15.23   
23      C   70.37                     284                    14.00   
24     GM   53.35                     374                    12.14   
25    WRK   46.23                     432                   -18.13   
26    AIV    4.89                    4089                     4.74   
27    GPS   25.83                     774                    -8.96   
28    APA   20.43                     978                    -0.95   
29     LB   59.49                     336                   -76.21   
30    HRB   20.55                     973                    19.51   
31    HPE   15.00                    1333                   -60.84   
32     GL   96.93                     206                    14.13   
33    LYV   95.63                     209                   -13.08   
34    KSS   57.00                     350                   -38.23   
35     CI  215.30                      92                     9.45   
36   DISH   34.00                     588                    11.28   
37   NWSA   24.50                     816                   -17.25   
38     BK   43.97                     454                    11.90   
39    STT   79.82                     250                    12.29   
40    PGR   89.21                     224                    10.27   
41   ZION   55.52                     360                     8.84   
42    CNC   60.22                     332                    19.64   
43    PHM   47.71                     419                     9.15   
44    OXY   28.06                     712                    -1.60   
45     GS  338.98                      59                    14.00   
46     RE  246.78                      81                    14.98   
47    TSN   69.18                     289                    12.52   
48    XOM   58.00                     344                   -10.86   
49    WHR  200.20                     N/A                    12.11   

   PE Percentile  Price-to-Book Ratio PB Percentile  Price-to-Sales Ratio  \
0       0.166337               0.5137      0.053465                0.4278   
1       0.110891             -56.4400      0.007921                0.1158   
2       0.156436               0.8920      0.071287                0.2859   
3       0.112871               0.5781      0.057426                0.8810   
4            0.2               0.6873      0.05

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [21]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [24]:
writer.save()